## Train an classifier for hyperspectral image


In [1]:
import os
from google.colab import drive
drive.mount('/content/drive')

path = "/content/drive/My Drive"

os.chdir(path)
os.listdir(path)

Mounted at /content/drive


['Indian_pines_gt.mat',
 'Indian_pines_corrected.mat',
 'Colab Notebooks',
 'data']

In [2]:
!pip install ColossalAI deepspeed

     |████████████████████████████████| 234 kB 13.3 MB/s 
     |████████████████████████████████| 517 kB 46.4 MB/s 
     |████████████████████████████████| 124 kB 50.0 MB/s 
     |████████████████████████████████| 108 kB 46.6 MB/s 
     |████████████████████████████████| 54 kB 2.8 MB/s 
     |████████████████████████████████| 18.2 MB 238 kB/s 
  Created wheel for deepspeed: filename=deepspeed-0.5.8-py3-none-any.whl size=532153 sha256=4d99aa004613b5a6dfc8efa283ec273b7ace5680c2baf1c1bea3e59f9e03cc07
  Stored in directory: /root/.cache/pip/wheels/8d/c6/82/cabd9a300c582a221591fd2c8c997e1f03f601e748aad44e4e
Successfully built deepspeed


In [3]:
!/opt/bin/nvidia-smi

Sat Dec  4 05:06:53 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8    29W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
import colossalai
from colossalai.engine import Engine, NoPipelineSchedule
from colossalai.trainer import Trainer
from colossalai.context import Config
import torch

Colossalai should be built with cuda extension to use the FP16 optimizer
Colossalai should be built with cuda extension to use the FP16 optimizer
apex is required for mixed precision training


First, we should initialize distributed environment. Though we just use single GPU in this example, we still need initialize distributed environment for compatibility. We just consider the simplest case here, so we just set the number of parallel processes to 1.

In [5]:
parallel_cfg = Config(dict(parallel=dict(
    data=dict(size=1),
    pipeline=dict(size=1),
    tensor=dict(size=1, mode=None),
)))
colossalai.init_dist(config=parallel_cfg,
          local_rank=0,
          world_size=1,
          host='127.0.0.1',
          port=8888,
          backend='nccl')

colossalai - torch.distributed.distributed_c10d - 2021-12-04 05:07:11,474 INFO: Added key: store_based_barrier_key:1 to store for rank: 0
colossalai - torch.distributed.distributed_c10d - 2021-12-04 05:07:11,476 INFO: Rank 0: Completed store-based barrier for key:store_based_barrier_key:1 with 1 nodes.
colossalai - torch.distributed.distributed_c10d - 2021-12-04 05:07:11,478 INFO: Added key: store_based_barrier_key:2 to store for rank: 0
colossalai - torch.distributed.distributed_c10d - 2021-12-04 05:07:11,480 INFO: Rank 0: Completed store-based barrier for key:store_based_barrier_key:2 with 1 nodes.
colossalai - torch.distributed.distributed_c10d - 2021-12-04 05:07:11,483 INFO: Added key: store_based_barrier_key:3 to store for rank: 0
colossalai - torch.distributed.distributed_c10d - 2021-12-04 05:07:11,485 INFO: Rank 0: Completed store-based barrier for key:store_based_barrier_key:3 with 1 nodes.


process rank 0 is bound to device 0


Load and normalize the CIFAR10 training and test datasets using `colossalai.nn.data`. Note that we have wrapped `torchvision.transforms`, so that we can simply use the config dict to use them.

In [6]:
transform_cfg = [
    dict(type='ToTensor'),
    dict(type='Normalize',
        mean=[0.4914, 0.4822, 0.4465],
        std=[0.2023, 0.1994, 0.2010]),
]

batch_size = 128

trainset = colossalai.nn.data.CIFAR10Dataset(transform_cfg, root='./data', train=True,download=True)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)
print(trainset)
testset = colossalai.nn.data.CIFAR10Dataset(transform_cfg, root='./data', train=False,download=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [7]:
!ls ./

'Colab Notebooks'   data   Indian_pines_corrected.mat   Indian_pines_gt.mat


In [13]:
import numpy as np
import torch
import torch.utils
import os
from tqdm import tqdm
from sklearn import preprocessing
from scipy import io, misc

# Load the image
folder_Pine = './'
img = io.loadmat(folder_Pine + 'Indian_pines_corrected.mat')
img = img['indian_pines_corrected']
gt = io.loadmat(folder_Pine + 'Indian_pines_gt.mat')['indian_pines_gt']
LABEL_VALUES = ["Undefined", "Alfalfa", "Corn-notill", "Corn-mintill",
                        "Corn", "Grass-pasture", "Grass-trees",
                        "Grass-pasture-mowed", "Hay-windrowed", "Oats",
                        "Soybean-notill", "Soybean-mintill", "Soybean-clean",
                        "Wheat", "Woods", "Buildings-Grass-Trees-Drives",
                        "Stone-Steel-Towers"]
IGNORED_LABELS = [0]
nan_mask = np.isnan(img.sum(axis=-1))
if np.count_nonzero(nan_mask) > 0:
  print("Warning: NaN have been found in the data. It is preferable to remove them beforehand. Learning on NaN data is disabled.")
img[nan_mask] = 0
gt[nan_mask] = 0
IGNORED_LABELS.append(0)
IGNORED_LABELS = list(set(IGNORED_LABELS))
# Normalization
img = np.asarray(img, dtype='float32')
#img = (img - np.min(img)) / (np.max(img) - np.min(img))
data = img.reshape(np.prod(img.shape[:2]), np.prod(img.shape[2:]))
#data = preprocessing.scale(data)
data  = preprocessing.minmax_scale(data)
img = data.reshape(img.shape)

# N_CLASSES = len(LABEL_VALUES) -  len(IGNORED_LABELS)
N_CLASSES = len(LABEL_VALUES)
# Number of bands (last dimension of the image tensor)
N_BANDS = img.shape[-1]

In [76]:
class HyperX(torch.utils.data.Dataset):
    """ Generic class for a hyperspectral scene """

    def __init__(self, data, gt, **hyperparams):
        """
        Args:
            data: 3D hyperspectral image
            gt: 2D array of labels
            patch_size: int, size of the spatial neighbourhood
            center_pixel: bool, set to True to consider only the label of the
                          center pixel
            data_augmentation: bool, set to True to perform random flips
            supervision: 'full' or 'semi' supervised algorithms
        """
        super(HyperX, self).__init__()
        self.data = data
        self.label = gt
        self.name = hyperparams['dataset']
        self.patch_size = hyperparams['patch_size']
        self.ignored_labels = set(hyperparams['ignored_labels'])
        self.flip_augmentation = hyperparams['flip_augmentation']
        self.radiation_augmentation = hyperparams['radiation_augmentation'] 
        self.mixture_augmentation = hyperparams['mixture_augmentation'] 
        self.center_pixel = hyperparams['center_pixel']
        supervision = hyperparams['supervision']
        # Fully supervised : use all pixels with label not ignored
        if supervision == 'full':
            mask = np.ones_like(gt)
            for l in self.ignored_labels:
                mask[gt == l] = 0
        # Semi-supervised : use all pixels, except padding
        elif supervision == 'semi':
            mask = np.ones_like(gt)
        x_pos, y_pos = np.nonzero(mask)
        p = self.patch_size // 2
        self.indices = np.array([(x,y) for x,y in zip(x_pos, y_pos) if x > p and x < data.shape[0] - p and y > p and y < data.shape[1] - p])
        self.labels = [self.label[x,y] for x,y in self.indices]
        np.random.shuffle(self.indices)
        
        #indices = self.indices
        #labels = self.labels
        #print("look at me !!!!!")
        #print(indices)
        #print(labels)
        
        #indices = [list(t) for t in zip(*indices)]
        #print(indices)
        #print(len(indices[0]))
        
        #data_new = np.zeros([len(labels),1,1,data.shape[2]])

        #print(data_new.shape)
        #data = data_new
        

        #targets = labels
        #print('targets is ' + str(len(targets)))
        #print("look at me !!!!!")

    #@staticmethod
    #def flip(*arrays):
    #    horizontal = np.random.random() > 0.5
    #    vertical = np.random.random() > 0.5
    #    if horizontal:
    #        arrays = [np.fliplr(arr) for arr in arrays]
    #    if vertical:
    #        arrays = [np.flipud(arr) for arr in arrays]
    #    return arrays

    #@staticmethod
    #def radiation_noise(data, alpha_range=(0.9, 1.1), beta=1/25):
    #    alpha = np.random.uniform(*alpha_range)
    #    noise = np.random.normal(loc=0., scale=1.0, size=data.shape)
    #    return alpha * data + beta * noise

    #def mixture_noise(self, data, label, beta=1/25):
    #    alpha1, alpha2 = np.random.uniform(0.01, 1., size=2)
    #    noise = np.random.normal(loc=0., scale=1.0, size=data.shape)
    #    data2 = np.zeros_like(data)
    #    for  idx, value in np.ndenumerate(label):
    #        if value not in self.ignored_labels:
    #            l_indices = np.nonzero(self.labels == value)[0]
    #            l_indice = np.random.choice(l_indices)
    #            assert(self.labels[l_indice] == value)
    #            x, y = self.indices[l_indice]
    #            data2[idx] = self.data[x,y]
    #    return (alpha1 * data + alpha2 * data2) / (alpha1 + alpha2) + beta * noise

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, i):
        print("look at me !!!!!")
        x, y = self.indices[i]
        x1, y1 = x - self.patch_size // 2, y - self.patch_size // 2
        x2, y2 = x1 + self.patch_size, y1 + self.patch_size

        data = self.data[x1:x2, y1:y2]
        label = self.label[x1:x2, y1:y2]
        

       # if self.flip_augmentation and self.patch_size > 1:
       #     # Perform data augmentation (only on 2D patches)
       #     data, label = self.flip(data, label)
       # if self.radiation_augmentation and np.random.random() < 0.1:
       #         data = self.radiation_noise(data)
       # if self.mixture_augmentation and np.random.random() < 0.2:
       #         data = self.mixture_noise(data, label)

        # Copy the data into numpy arrays (PyTorch doesn't like numpy views)
        data = np.asarray(np.copy(data)), dtype='float32')
        label = np.asarray(np.copy(label), dtype='int64')
        targets = np.asarray(np.copy(label), dtype='int64')

        data = np.expand_dims(data, axis=0)
        targets = targets.tolist()

        #indices = self.indices
        #labels = self.labels
        #indices = [list(t) for t in zip(*indices)]
        #data_new = np.zeros([len(labels),1,1,data.shape[2]])
        #data = data_new
        #targets = labels

        # Load the data into PyTorch tensors
        data = torch.from_numpy(data)
        label = torch.from_numpy(label)
        targets = torch.from_numpy(targets)

        ## Extract the center label if needed
        #if self.center_pixel and self.patch_size > 1:
        #    label = label[self.patch_size // 2, self.patch_size // 2]
        ## Remove unused dimensions when we work with invidual spectrums
        #elif self.patch_size == 1:
        #    data = data[:, 0, 0]
        #    label = label[0, 0]

        # Add a fourth dimension for 3D CNN
        #if self.patch_size > 1:
        #    # Make 4D data ((Batch x) Planes x Channels x Width x Height)
        #    data = data.unsqueeze(0)
       
        print(data.size())
        print(label.size())    
        
        return data, label



In [70]:
import sklearn.model_selection
import torch.utils.data as data
import numpy as np
from scipy import io
hyperparams = {'batch_size': 1, 
        'dataset': 'IndianPines',
        'patch_size': 1,
        'ignored_labels': [0],
        'flip_augmentation': False,
        'radiation_augmentation': False, 
        'mixture_augmentation': False,
        'center_pixel': True,
        'supervision': 'full'
        }

def sample_gt(gt, train_size, mode='random'):
    """Extract a fixed percentage of samples from an array of labels.
    Args:
        gt: a 2D array of int labels
        percentage: [0, 1] float
    Returns:
        train_gt, test_gt: 2D arrays of int labels
    """
    
    
#    print(gt)
#    gt = gt.get('indian_pines_gt')
    print(gt)
    print(gt.shape)
    
    indices = np.nonzero(gt)
    print(indices)
    
    #print(X)
    #for i in range(len(X))
    #  y = []
    #  y[i-1] = gt(indices[0][i],indices[1][i])  # classes
    #  y = np.array(y)

    X = list(zip(*indices)) # x,y features #将x轴和y轴的索引坐标合为一个二维坐标，共N个坐标，N为有标签的像元的数目（标签非0）。
    print(X)

    y1 = gt[indices]
    y = y1.ravel() # classes #将N个非零标签像元对应的标签去除并组成一个列表。
    #ravel()方法将数组维度拉成一维数组


    train_gt = np.zeros_like(gt) 
    test_gt = np.zeros_like(gt)
    if train_size > 1:
       train_size = int(train_size)
    
    if mode == 'random':
       train_indices, test_indices = sklearn.model_selection.train_test_split(X, train_size=train_size, stratify=y)
       train_indices = [list(t) for t in zip(*train_indices)]
       test_indices = [list(t) for t in zip(*test_indices)]
       train_gt[train_indices] = gt[train_indices]
       test_gt[test_indices] = gt[test_indices]
    return train_gt, test_gt




#TRAIN_GT = './' 
#TEST_GT = './'
#train_gt = io.loadmat(TRAIN_GT+'Indian_pines_gt.mat')
#test_gt = io.loadmat(TEST_GT+'Indian_pines_gt.mat')
train_gt, test_gt = sample_gt(gt, 0.1, mode='random')

# Generate the dataset
train_dataset = HyperX(img, train_gt, **hyperparams)
test_dataset = HyperX(img, test_gt, **hyperparams)    
print("HSI train dataset")
print(train_dataset)
##用来把训练数据分成多个小组，此函数每次抛出一组数据。直至把所有的数据都抛出。就是做一个数据的初始化。
train_loader = data.DataLoader(train_dataset,
                batch_size=hyperparams['batch_size'],
                shuffle=True)
test_loader = data.DataLoader(test_dataset,
               batch_size=hyperparams['batch_size'])
#PyTorch中数据读取的一个重要接口是torch.utils.data.DataLoader，该接口定义在dataloader.py脚本中，
#只要是用PyTorch来训练模型基本都会用到该接口，该接口主要用来将自定义的数据读取接口的输出或者PyTorch已有的数据读取接口的输入按照batch size封装成Tensor，
#后续只需要再包装成Variable即可作为模型的输入，因此该接口有点承上启下的作用，比较重要。
      


[[3 3 3 ... 0 0 0]
 [3 3 3 ... 0 0 0]
 [3 3 3 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
(145, 145)
(array([  0,   0,   0, ..., 143, 143, 143]), array([ 0,  1,  2, ..., 30, 31, 32]))
[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (0, 10), (0, 11), (0, 12), (0, 13), (0, 14), (0, 15), (0, 16), (0, 17), (0, 18), (0, 19), (0, 21), (0, 22), (0, 23), (0, 71), (0, 72), (0, 73), (0, 74), (0, 75), (0, 76), (0, 77), (0, 78), (0, 79), (0, 80), (0, 81), (0, 82), (0, 83), (0, 84), (0, 85), (0, 86), (0, 87), (0, 88), (0, 89), (0, 90), (0, 91), (0, 92), (0, 93), (0, 94), (0, 97), (0, 98), (0, 99), (0, 100), (0, 101), (0, 102), (0, 103), (0, 104), (0, 105), (0, 106), (0, 107), (0, 108), (0, 109), (0, 110), (0, 111), (0, 112), (0, 113), (0, 114), (0, 115), (0, 116), (0, 117), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (1, 10), (1, 11), (1, 12), (1, 13), (1, 14), (1, 15), (1, 16), (1, 17), (1, 18), (1, 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:57: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:58: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.


We just define a simple network here.

In [71]:
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import init

class HSI_processing(nn.Module):
    """
    Simple NN-based network
    """
    @staticmethod
    def weight_init(m):
        if isinstance(m, nn.Linear):
            init.kaiming_normal_(m.weight)
            init.zeros_(m.bias)

    def __init__(self, input_channels=200, n_classes=17, dropout=False):
        super(HSI_processing, self).__init__()
        self.use_dropout = dropout
        if dropout:
            self.dropout = nn.Dropout(p=0.5)
       
        self.fc1 = nn.Linear(input_channels, 2048)
        self.fc2 = nn.Linear(2048, 4096)
        self.fc3 = nn.Linear(4096, 2048)
        self.fc4 = nn.Linear(2048, n_classes)

        self.apply(self.weight_init)

    def forward(self, x):
        print(x.size())
        x = x.view(x.shape[0], -1) #(200,1)
        print(x.size())
        x = x.t()          #(1,200)
        print(x.size())
        x = F.relu(self.fc1(x))   #(1,2048)
        print(x.size())
        if self.use_dropout:
            x = self.dropout(x)
        x = F.relu(self.fc2(x))
        print(x.size())
        if self.use_dropout:
            x = self.dropout(x)
        x = F.relu(self.fc3(x))
        print(x.size())
        if self.use_dropout:
            x = self.dropout(x)
        x = self.fc4(x)
        print(x.size())
        return x  #(1,17)

model = HSI_processing().cuda()

Define a Loss function and optimizer. And then we use them to initialize `Engine` and `Trainer`. We provide various training / evaluating hooks. In this case, we just use the simplest hooks which can compute and print loss and accuracy.

In [72]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss()
schedule = NoPipelineSchedule()

engine = Engine(
        model=model,
        criterion=criterion,
        optimizer=optimizer,
        lr_scheduler=None,
        schedule=schedule
    )
trainer = Trainer(engine=engine,
          hooks_cfg=[dict(type='LossHook'), dict(type='LogMetricByEpochHook'), dict(type='AccuracyHook')],
          verbose=True)

colossalai - rank_0 - 2021-12-04 06:11:19,095 WARNING: No gradient handler is set up, please make sure you do not need to all-reduce the gradients after a training step.
colossalai - rank_0 - 2021-12-04 06:11:19,100 INFO: build LogMetricByEpochHook for train, priority = 1
colossalai - rank_0 - 2021-12-04 06:11:19,102 INFO: build LossHook for train, priority = 10
colossalai - rank_0 - 2021-12-04 06:11:19,105 INFO: build AccuracyHook for train, priority = 10


Then we set training configs. We train our model for 100 epochs and it will be evaluated every 10 epoch. Set `display_progress` to `True` to display the training / evaluating progress bar.

In [77]:
num_epochs = 100
test_interval = 10
trainer.fit(
        train_dataloader=train_loader,
        test_dataloader=test_loader,
        max_epochs=num_epochs,
        display_progress=True,
        test_interval=test_interval
    )

[Epoch 2 train]:   0%|          | 0/1013 [00:00<?, ?it/s]

torch.Size([200])
torch.Size([])
torch.Size([200])
torch.Size([200, 1])
torch.Size([1, 200])
torch.Size([1, 2048])
torch.Size([1, 4096])
torch.Size([1, 2048])
torch.Size([1, 17])


ValueError: ignored